# Installation des dépendances

Pour pouvoir exécuter ce Notebook, l'environnement de développement doit être bien configuré. Puisque dans ce POC nous utilisons YoloV5, il faut installer de nombreuses dépendances dont PyTorch.

<div class="alert alert-info">
    Afin d'exécuter ce Notebook sur AWS SageMaker, il faut utiliser le kernel <code>conda_python3</code>.
</div>

<div class="alert alert-info">
    Il faut compter environ 5 minutes pour l'installation des dépendances.
</div>

In [ ]:
!git clone https://github.com/rkuo2000/yolov5

On installe ensuite les dépendances pour YoloV5.

In [ ]:
!pip install -q -r yolov5/requirements.txt

Enfin, nous installons les dépendances liées aux frameworks Deep Learning (ici PyTorch).

<div class="alert alert-warning">
    Attention, il faut utiliser la bonne version de PyTorch : sur SageMaker, c'est la version <code>1.11.0+cu113</code> mais sur un autre environment, cela peut être différent !
</div>

In [ ]:
!pip install -q \
    torch==1.11.0+cu113 \
    torchvision==0.12.0+cu113 \
    torchaudio==0.11.0 \
    gcc7 \
    opendatasets \
    pycocotools \
    split-folders \
    --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
# Ceci est une petite correction nécessaire pour YoloV5
!pip install -q --upgrade scipy

<div class="alert alert-info">
    Il est nécessaire de créer un compte sur <a href="https://kaggle.com" target="_blank">Kaggle</a> pour télécharger les données.
</div>

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/datasets/kneroma/tacotrashdataset")

# Préparation des données

Le but de cette partie est de **transformer le fichier d'annotations JSON en plusieurs fichier texte** que le modèle YoloV5 va utiliser pour s'entraîner et notamment apprendre quelles bounding boxes il doit être capable de retrouver.

Dans ce format, il y a ainsi **un fichier texte par image** où pour chaque déchet dans l'image est associée une ligne contenant l'identifiant du déchet et les coordonnées de la bounding box. Dans l'exemple suivant, l'image associée contiendrait 2 types de déchets (identifiants 4 et 7) et 3 déchets présents sur l'image.

In [ ]:
from pycocotools.coco import COCO

# On importe le fichier des annotations sous la forme d'un objet COCO
data_source = COCO(annotation_file="./tacotrashdataset/data/annotations.json")

On extrait ensuite les identifiants des images, des catégories et des super catégories.

### ➡️ À toi de jouer

Créer les objets Python suivants.

- `categories` est une lmiste contient l'ensemble des catégories avec un identifiant (`id`), le nom de la catégorie (`name`) et le nom de la super catégorie (`supecategory`). Par exemple, on aura `[{'supercategory': 'Aluminium foil', 'id': 0, 'name': 'Aluminium foil'}, ...]`.
- `classes_num` est un dictionnaire qui indique l'identifiant (`id`) de n'importe quel catégorie. Par exemple, on aura `{'Aluminium foil': 0, 'Battery': 1, ...}`.
- `coco_labels` est un dictionnaire qui contient, pour chaque clé incrémentale, l'identifiant de la catégorie. Par exemple, on aura `{0: 0, 1: 1, ...}`. On utilise ce dictionnaire si par exemple on ne souhaite pas utiliser 100% des catégories disponibles, mais uniquement une partie.
- `coco_labels_inverse` est le dictionnaire inverse de `coco_labels` : si `coco_labels[A] = B`, alors `coco_labels_inverse[B] = A`.

In [ ]:
img_ids = data_source.getImgIds()
catIds = data_source.getCatIds()
categories = data_source.loadCats(catIds)
# Trier les catégories selon l'ID
# TODO
categories

# ...

classes = {}
coco_labels = {}
coco_labels_inverse = {}

# Pour chaque catégorie
for c in categories:
    # Remplir les variables coco_labels, coco_labels_inverse et classes
    # TODO
    # ...

class_num = {}

On peut vérifier que l'on dipose de toutes les informations en affichant une dizaine de super catégories.

In [ ]:
categories[:10]

De même pour les catégories.

In [ ]:
classes

Si tout est bon, nous pouvons alors construire les fichiers texte.

### ➡️ À toi de jouer

Pour chaque image, nous allons créer un fichier texte qui contient les annotations. Pour cela, nous allons donc utiliser la variable `coco_labels_inverse` pour chaque ligne d'annotation, et remplir le reste de la ligne avec la bounding box associée.

In [ ]:
# Nous créons un dossier temporaire pour y mettre les images et les fichiers texte
!mkdir -p tmp/labels tmp/images

save_base_path  = 'tmp/labels/'
save_image_path = 'tmp/images/'

# Cette boucle permet d'extraire le contenu de chaque fichier texte à partir du fichier annotation et de créer le fichier
for index, img_id in tqdm.tqdm(enumerate(img_ids), desc="Conversion fichier JSON en fichier texte"):
    # Pour chaque image on extrait le nom et ses dimensions
    img_info = data_source.loadImgs(img_id)[0]
    save_name = img_info['file_name'].replace('/', '_')
    file_name = save_name.split('.')[0]
    height = img_info['height']
    width = img_info['width']
    
    # On créé le fichier texte
    save_path = save_base_path + file_name + '.txt'
    is_exist = False 
    
    # On remplit le fichier
    with open(save_path, mode='w') as fp:
        # Extraction de l'identifiant de la catégorie
        annotation_id = data_source.getAnnIds(img_id)
        boxes = np.zeros((0, 5))
        if len(annotation_id) == 0: 
            fp.write('')
            continue
            
        # Récupération des bounding boxes
        annotations = data_source.loadAnns(annotation_id)
        lines = ''  
        for annotation in annotations:
            # Pour chaque annotation, récupérer le label depuis coco_labels_inverse et ajouter les bounding box sur la ligne.
            # TODO
            # ...
            
        fp.writelines(lines)
        
    # Si tout est OK, on enregistre dans le dossier data
    if is_exist:
        shutil.copy('./tacotrashdataset/data/{}'.format(img_info['file_name']), os.path.join(save_image_path, save_name))
    else:
        os.remove(save_path)

Pour terminer, il ne reste plus qu'à découper le dossier en trois sous-dossiers, qui vont représenter nos sous-ensembles d'entraînement, de test et de validation.

In [ ]:
import splitfolders

splitfolders.ratio("tmp", output="taco", ratio=(0.8, 0.1,0.1))

Au final, notre dossier `taco/` va contenir nos trois dossiers : nous pouvons alors supprimer le dossier temporaire `tmp/`.

In [ ]:
!rm -rf ./tmp